# Lab 7 - Model comparison

F - number of letters in first name L - number of letters in last name N = (L+F)*100

In [1]:
F = len("JINHUNG")
L = len("TOH")
N = (L+F)*100

### Excercise 1 - generate data
1. Compile code_1.stan and code_2.stan
2. Generate data for rest of excercises.

In [34]:
from cmdstanpy import CmdStanModel
import arviz as az
import numpy as np
import scipy.stats as stats
import pandas as pd
import statistics as s 

import matplotlib.pyplot as plt
import matplotlib as mpl

In [5]:
code1 = CmdStanModel(stan_file='code_1.stan')
data1 = code1.sample(data={'N': N}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


In [6]:
df1 = data1.draws_pd()
df1

,lp__,accept_stat__,theta,y[1],y[2],y[3],y[4],y[5],y[6],y[7],...,y[991],y[992],y[993],y[994],y[995],y[996],y[997],y[998],y[999],y[1000]
0,0.0,0.0,0.01,0.535499,0.826999,0.145525,0.514284,-1.14738,-0.366778,-2.26815,...,0.180948,-1.69684,-1.77174,0.715475,0.33441,-2.59767,-0.499751,1.27932,-0.357928,-0.638488


In [7]:
code2 = CmdStanModel(stan_file='code_2.stan')
data2 = code2.sample(data={'N': N}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


In [8]:
df2 = data2.draws_pd()
df2

,lp__,accept_stat__,"X[1,1]","X[2,1]","X[3,1]","X[4,1]","X[5,1]","X[6,1]","X[7,1]","X[8,1]",...,y[991],y[992],y[993],y[994],y[995],y[996],y[997],y[998],y[999],y[1000]
0,0.0,0.0,0.734529,0.579883,-1.08972,-1.54368,1.80312,-0.825802,-1.28514,1.54254,...,1.30023,0.566172,3.02075,2.57167,-0.252406,-3.98776,-2.57042,-2.13398,2.28262,-1.12381


### Excercise 2 - compare normal and student models for data from first file
1. Compile both models
2. Fit both models
3. Using az.compare and az.plot_compare analyze both models using loo and waic criteria.

In [28]:
code3 = CmdStanModel(stan_file='code_3.stan')
code4 = CmdStanModel(stan_file='code_4.stan')

y_data1 = df1.iloc[0,3:]
y_data2 = df2.iloc[0,3005:]

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:found newer exe file, not recompiling


In [52]:
#Code3 stan model with data1
code3_fit_data1 = code3.sample(data={'N': N, 'y':y_data1}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)

#Convertion to arviz InferenceData
arviz_code3_fit_data1 = az.from_cmdstanpy(
    posterior=code3_fit_data1,
    posterior_predictive=["log_lik","y_hat"],
    observed_data={"y": y_data1},
)

#Code4 stan model with data1
code4_fit_data1 = code4.sample(data={'N': N, 'y':y_data1}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)
                            
#Convertion to arviz InferenceData
arviz_code4_fit_data1 = az.from_cmdstanpy(
    posterior=code4_fit_data1,
    posterior_predictive=["log_lik","y_hat"],
    observed_data={"y": y_data1},
)

INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.
INFO:cmdstanpy:CmdStan start processing


chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


In [53]:
compare_data1_dict = {"Code3": arviz_code3_fit_data1, "Code4": arviz_code4_fit_data1}
az.compare(compare_data1_dict)

C:\Users\User\anaconda3\lib\site-packages\arviz\stats\stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(


TypeError: log likelihood not found in inference data object

In [49]:
code3_fit_data2 = code3.sample(data={'N': N, 'y':y_data2}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)

code4_fit_data2 = code4.sample(data={'N': N, 'y':y_data2}, fixed_param=True, 
                            iter_sampling=1, 
                            iter_warmup=1000, 
                            chains = 1)

INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.
INFO:cmdstanpy:CmdStan start processing


chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


In [54]:
import arviz as az

compare_dict = {"non centered": data1, "centered": data2}
az.compare(compare_dict)

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
non centered,0,-30.687290,0.841888,0.000000,1.0,1.364996,0.000000,False,log
centered,1,-30.810374,0.954053,0.123084,0.0,1.426837,0.086046,False,log


In [55]:
data1

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In [56]:
data2

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In [57]:
arviz_code4_fit_data1

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> observed_data